## IMDB Review Exploration & Gated Recurrent Unit (GRU) *Mood* Model

[This is a dataset of 25,000 movies reviews from IMDB](https://ai.stanford.edu/%7Eamaas/data/sentiment/), labeled by sentiment (positive/negative). Reviews have been preprocessed, and each review is encoded as a list of word indexes (integers). For convenience, words are indexed by overall frequency in the dataset.

As a convention, "0" does not stand for a specific word, but instead is used to encode the pad token.

[Documentation](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/imdb/load_data) | [Paper: Learning Word Vectors for Sentiment Analysis](https://ai.stanford.edu/%7Eamaas/papers/wvSent_acl2011.pdf)

In [401]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense
import numpy as np

In [402]:
word_index = imdb.get_word_index() # Just a dictionary, mapping each token to its word

In [403]:
len(word_index)

88584

That's a lotta unique words.

### Top 40 most frequent words

The word indices in the dataset are offset by 4, for special tokens:

`0` For encoding padding token, as mentioned<br>
`1` Start of sequence<br>
`2` Unknown word (OOV or UNK, "out-of-vocabulary" and "unknown")<br>
`3` Index actual words with this index and higher.

In [404]:
pad_char = 0
start_char = 1
oov_char = 2
index_from = 3

inverted_word_index = dict(
    (i + index_from, word) for (word, i) in word_index.items()
)
# Update `inverted_word_index` to include `start_char` and `oov_char`
inverted_word_index[start_char] = "[START]"
inverted_word_index[oov_char] = "[UNK]"

In [405]:
for i in range(20):
    print(f"{inverted_word_index.get(i+4)}, ", end='')

the, and, a, of, to, is, br, in, it, i, this, that, was, as, for, with, movie, but, film, on, 

And that's a lotta stop words. I think we should remove most of them before we train.<br>
But before we do that, let's check out the data some more.

All of the top words make sense from a natural language perspective except one. Probably caused by the html tag `<br>`?<br>
I cannot think of any other reason *br* would be in the reviews. Let's check the next 20 words.

In [406]:
for i in range(20):
    print(f"{inverted_word_index.get(24+i)}, ", end='')

not, you, are, his, have, he, be, one, all, at, by, an, they, who, so, from, like, her, or, just, 

Seems fine. Let's check out the reviews.

In [407]:
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

In [408]:
decoded_sentence = ' '.join(inverted_word_index.get(i, '?') for i in train_data[np.random.randint(0, len(train_data))])
decoded_sentence

"[START] the marriage of maria [UNK] [UNK] is about a german girl maria getting married to a german soldier herman [UNK] just at the ending of the war after being married for half a day and a night herman is send to the front again to make ends meet maria starts working at a bar for mainly american soldiers and get to know a black soldier she got word that herman died at the front and things develop between her and the american soldier herman walks in on them in bed and after a confrontation between him and the american maria killed the american herman admits to the murder ends up in jail and maria [UNK] to wait for him the country is in [UNK] one sees people leaving everything that they are busy with for a cigarette there are food [UNK] it is in short a time of survival of the [UNK] br br basically this film projects [UNK] attitudes those attitudes she [UNK] herself under the mentioned circumstances as a metaphor for [UNK] loss of soul after they lost the war and how it proceeds to [U

It's not really readable like this, and I want to continue EDA of the reviews, so I made a function to fetch random samples of the train data and make it a bit more readable.

In [409]:
def get_random_sample(n=25):
    rng = np.random.randint(0, len(train_data))
    review = ' '.join(inverted_word_index.get(i, '?') for i in train_data[rng])
    sentiment = train_labels[rng]
    words = review.split()
    lines = [' '.join(words[i:i+n]) for i in range(0, len(words), n)]
    result = '\n'.join(lines)

    print(f"Sample {rng}:\n\n{result} \n\nLabel: {sentiment}")

get_random_sample()

Sample 24954:

[START] [UNK] is actually a step up in terms of what the original movie was in story and in special effects jason scott lee is
good as a vampire hunter looking for the count himself if you remember him he was from the movie dragon the bruce lee story '
jason london is funny as luke the kid who helps the woman he loves from a far steal [UNK] body from the [UNK] diane [UNK]
is good as the woman who steals [UNK] body in order to finds a cure for her dying boyfriend and stephen [UNK] is great as
dracula himself giving a better performance than gerard butler did the count in the original film roy [UNK] rounds out the rest of the cast
in this movie and he does a decent job as the mentor of jason scott lee's character this is the second sequel in the trilogy
and they are off to a good start it's up in the air whether the last film will close the series out on a good
note 

Label: 0


In [410]:
get_random_sample()

Sample 21468:

[START] i remember watching the bsg pilot i can describe that night exactly i remember what chair i sat in that show was magic it
came alive i enjoyed the first two years of bsg i enjoyed parts of the third year even and i watched every episode of the
fourth year totally [UNK] in great hopes that it would somehow turn around well it didn't br br i watched the caprica pilot and was
enthralled there was hope for something good here then i started watching the regular episodes and they are getting more and more boring br br
it's too obvious too predictable it reminds me of the [UNK] political correctness of his last failed show [UNK] br br much of his line
work on [UNK] was good when he focused on bsg in an organized way it was good this was especially true early on when they
more or less followed the pattern of episodes set by the first bsg series when they [UNK] from that after meeting up with [UNK] cain
and the [UNK] it all went to pot it was like he wrote the rest of th

In [411]:
get_random_sample()

Sample 4822:

[START] this independent film was one of the best films at the tall grass film festival that i have ever seen there i loved it
there are so many things that was great about the film on top of all that the cast and crew that i had the opportunity
to meet were absolutely phenomenal i thought that [UNK] did a great job in his role and ricky [UNK] was absolutely true to his role
for a disney actor i was amazed at his talent to be able to go from cheesy teen comedy to such an adult role with
no problems the talent in the film was just amazing the cinematography was just great if you want to see an independent film this is
one really that you should see i think that mr [UNK] would have been so proud to have such a [UNK] in his festival and
his parents loved the movie so much when it won the audience favorite they went and saw it again this truly was a great film
it was dark and funny and sad and truly emotional it was just fabulous i am honestly just so [UNK] by this film and i

Fun read! Let's get cooking.

## Removing stop words

We could use [nltk](https://www.nltk.org), but since we have this slick API we might as well just fetch the data as we want it.

In [412]:
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=20040, skip_top=16, maxlen=200, start_char=1, oov_char=2)

I cut the most common words at index 16, becuase I want to include the conjunction "but", since it can add contrast, contrary to the more agreeing "and".
Not sure if it's a good idea, but let's try!

Well, that was easy "cleaning".

## GRU Model & Training

In [413]:
model = Sequential()
model.add(Embedding(input_dim=20040, output_dim=128, input_length=200))
model.add(GRU(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_19 (Embedding)    (None, 200, 128)          2565120   
                                                                 
 gru_19 (GRU)                (None, 64)                37248     
                                                                 
 dense_19 (Dense)            (None, 1)                 65        
                                                                 
Total params: 2602433 (9.93 MB)
Trainable params: 2602433 (9.93 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [414]:
train_data = pad_sequences(train_data, maxlen=200)
test_data = pad_sequences(test_data, maxlen=200)

model.fit(train_data, train_labels, epochs=4, batch_size=64, validation_split=0.2)
test_loss, test_acc = model.evaluate(test_data, test_labels)

Epoch 1/4
179/179 [==============================] - 34s 172ms/step - loss: 0.5246 - accuracy: 0.7278 - val_loss: 0.3431 - val_accuracy: 0.8491
Epoch 2/4
179/179 [==============================] - 31s 171ms/step - loss: 0.2404 - accuracy: 0.9064 - val_loss: 0.3227 - val_accuracy: 0.8649
Epoch 3/4
179/179 [==============================] - 31s 171ms/step - loss: 0.1345 - accuracy: 0.9530 - val_loss: 0.3649 - val_accuracy: 0.8638
Epoch 4/4
459/459 [==============================] - 10s 21ms/step - loss: 0.4559 - accuracy: 0.8371


## Inference

In [415]:
new_reviews1 = [
"really enjoyed movie, acting was fantastic but plot was not so great",
"really enjoyed movie, acting was fantastic plot was not so great",
"the, and, a, of, to, is, br, in, it, i, this, that, was, as, for, with, movie, but, film, on, "]

new_reviews2 = [
"It was just decent. I would not watch it again, I could have it running in the background without changing channel",
"It was just decent. I would not watch it again, but I could have it running in the background without changing channel",
"just decent. would not watch again could have running background without changing channel",
"just decent. would not watch again but could have running background without changing channel"]

gpt_positive_reviews = [
"Indulging in a can of tomato soup has never been so satisfying! This culinary delight is a true game-changer, offering a burst of rich, velvety flavor that instantly warms the soul. The vibrant red hue is a testament to the quality of the tomatoes used, providing a visual feast before the first spoonful. The texture is perfectly smooth, creating a luscious and comforting consistency that feels like a hug in a bowl. The harmonious blend of herbs and spices adds layers of complexity to the taste, striking a delightful balance between savory and subtly sweet notes. The aroma that wafts from the steaming soup is a prelude to the culinary masterpiece that awaits. Whether enjoyed as a quick solo lunch or paired with a grilled cheese sandwich for a heartier meal, this tomato soup elevates any dining experience. The convenience of a ready-to-eat can makes it a go-to option for busy days, delivering gourmet quality in the blink of an eye. With a maximum sequence of 200, it's challenging to convey the full depth of the sensory journey that this tomato soup offers. In summary, this can of tomato soup is a must-have pantry essential, delivering a palate-pleasing experience that brings comfort and joy to every spoonful.",
"Riding the tandem bike was an absolute delight from start to finish! The smooth coordination between two riders provided an exhilarating experience that brought joy and laughter throughout the journey. The comfortable seating and ergonomic design made every pedal stroke effortless, allowing us to explore scenic routes without any discomfort. The sturdy frame and responsive handling ensured a safe and secure ride, while the efficient gearing system effortlessly tackled various terrains. With a generous sequence length of 200, this tandem bike review barely scratches the surface of our amazing adventures. Whether cruising along coastal paths or conquering challenging hills, the tandem bike proved to be the perfect companion for creating lasting memories and sharing unforgettable moments. Overall, a fantastic ride that exceeded all expectations!"]

gpt_negative_reviews = [
"Opening the can of tomato soup was a disappointing experience from the very start. The promise of a delightful culinary treat quickly faded as I encountered an overwhelmingly bland aroma that failed to evoke the richness one expects from a tomato soup. Upon tasting, the lackluster flavor profile became painfully apparent – a thin and insipid broth with an absence of the robust tomato essence. The texture further added to the dissatisfaction, as it lacked the desired creaminess and depth that one anticipates in a quality tomato soup. Instead of a comforting consistency, the soup felt watery and devoid of the velvety smoothness that should accompany such a classic dish. The touted blend of herbs and spices turned out to be a mere whisper, contributing little to enhance the overall taste. While convenience is a selling point for canned soups, this particular product left much to be desired. The aftertaste was distinctly metallic, leaving an unpleasant lingering sensation. With a maximum sequence length of 200, it's challenging to convey the full extent of disappointment, but suffice it to say, this can of tomato soup failed to meet even the most basic expectations. A regrettable choice that falls far short of the culinary experience one would hope for.",
"The tandem bike I recently purchased turned out to be a regrettable investment, failing to live up to even the most modest expectations. From the outset, the design proved to be cumbersome and impractical, making the initial setup an arduous task. The promised seamless coordination between riders was anything but, as the tandem bike exhibited an alarming lack of responsiveness to even the most synchronized pedaling efforts. Comfort, a crucial factor in any bike ride, was sorely lacking. The seating arrangement was far from ergonomic, causing discomfort and fatigue within a short period. Instead of a smooth and enjoyable ride, the tandem bike delivered a jarring and unpleasant experience, amplifying every bump and uneven surface on the road. The build quality left much to be desired, with noticeable issues in the frame that raised concerns about safety. Rather than instilling confidence, the tandem bike inspired a constant worry about its structural integrity, overshadowing any attempt to appreciate the outdoor scenery. As for the promised sense of unity and shared enjoyment, it was nowhere to be found. The tandem bike, far from fostering a sense of togetherness, ended up causing frustration and tension between riders. With a maximum sequence length of 200, it's challenging to encapsulate the full extent of disappointment, but suffice it to say, this tandem bike failed to deliver on its promises and left me questioning the decision to purchase it. A regrettable choice for anyone seeking a harmonious and enjoyable biking experience."
]

In [416]:
def predict(reviews):
    for i in range(len(reviews)):
        new_text = reviews[i]

        new_sequence = [imdb.get_word_index().get(word, 0) for word in new_text.split()]
        new_sequence = [index if index < 20040 else 0 for index in new_sequence]
        padded = pad_sequences([new_sequence], maxlen=200)
        prediction = model.predict(padded)

        sentiment = "Positive" if prediction[0, 0] > 0.5 else "Negative"
        print(new_sequence)
        print(reviews[i])
        print(f"Predicted sentiment: {sentiment} (Confidence: {prediction[0, 0] * 100:.2f}%)")
        print()

predict(new_reviews1)

1/1 [==============================] - 0s 322ms/step
[63, 507, 0, 113, 13, 774, 18, 111, 13, 21, 35, 84]
really enjoyed movie, acting was fantastic but plot was not so great
Predicted sentiment: Positive (Confidence: 93.43%)

1/1 [==============================] - 0s 39ms/step
[63, 507, 0, 113, 13, 774, 111, 13, 21, 35, 84]
really enjoyed movie, acting was fantastic plot was not so great
Predicted sentiment: Positive (Confidence: 92.59%)

1/1 [==============================] - 0s 39ms/step
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
the, and, a, of, to, is, br, in, it, i, this, that, was, as, for, with, movie, but, film, on, 
Predicted sentiment: Positive (Confidence: 80.39%)



Third review is like a 'baseline' test? Maybe..? The model has a bias towards positive reviews?

But the first and second are interesting, as in how the word `but` can have an impact. Although we shouldn't jump to conclusions just by this tiny cherry pick.
I guess it's is more of a wildcard. Unlesss we can feature engineer it somehow, I'm not sure the connotation following the word `but` tend to be positive or negative, I mean intuitively it's usually followed by something negative? Maybe it's best just consider it a stop word.

In [417]:
predict(new_reviews2)

1/1 [==============================] - 0s 39ms/step
[0, 13, 40, 0, 0, 59, 21, 103, 9, 0, 0, 97, 25, 9, 617, 8, 1, 975, 206, 2543, 1305]
It was just decent. I would not watch it again, I could have it running in the background without changing channel
Predicted sentiment: Positive (Confidence: 72.19%)

1/1 [==============================] - 0s 42ms/step
[0, 13, 40, 0, 0, 59, 21, 103, 9, 0, 18, 0, 97, 25, 9, 617, 8, 1, 975, 206, 2543, 1305]
It was just decent. I would not watch it again, but I could have it running in the background without changing channel
Predicted sentiment: Positive (Confidence: 74.56%)

1/1 [==============================] - 0s 39ms/step
[40, 0, 59, 21, 103, 171, 97, 25, 617, 975, 206, 2543, 1305]
just decent. would not watch again could have running background without changing channel
Predicted sentiment: Positive (Confidence: 77.20%)

1/1 [==============================] - 0s 39ms/step
[40, 0, 59, 21, 103, 171, 18, 97, 25, 617, 975, 206, 2543, 1305]
just decent. w

But to no surpise, removing the stop words from input improves confidence.

I will test two negative and two positive reviews from something entirely unrelated.

In [418]:
predict(gpt_positive_reviews)

1/1 [==============================] - 0s 38ms/step
[0, 8, 3, 67, 4, 10370, 5769, 44, 112, 74, 35, 0, 0, 0, 3034, 6, 3, 280, 0, 3988, 3, 5563, 4, 0, 0, 6894, 12, 3502, 18395, 1, 0, 0, 5752, 764, 0, 6, 3, 4961, 5, 1, 486, 4, 1, 4805, 0, 3757, 3, 1111, 6529, 156, 1, 83, 0, 0, 10027, 6, 947, 0, 1852, 3, 11030, 2, 12989, 10685, 12, 761, 37, 3, 9141, 8, 3, 0, 0, 0, 3887, 4, 0, 2, 0, 1605, 5885, 4, 4637, 5, 1, 0, 3344, 3, 1914, 2969, 197, 0, 2, 6081, 1044, 0, 0, 0, 12, 0, 36, 1, 8697, 5769, 6, 3, 18696, 5, 1, 0, 988, 12, 0, 0, 507, 14, 3, 1602, 4318, 6800, 39, 8597, 16, 3, 0, 3040, 15457, 15, 3, 0, 0, 11, 10370, 5769, 12142, 98, 13106, 0, 0, 11719, 4, 3, 0, 67, 163, 9, 3, 0, 5447, 15, 2955, 0, 4495, 0, 486, 8, 1, 9374, 4, 32, 0, 0, 3, 7523, 717, 4, 0, 42, 4736, 5, 2830, 1, 365, 1134, 4, 1, 0, 1308, 12, 11, 10370, 5769, 0, 0, 0, 11, 67, 4, 10370, 5769, 6, 3, 0, 0, 0, 4495, 3, 0, 582, 12, 958, 5093, 2, 1802, 5, 172, 0]
Indulging in a can of tomato soup has never been so satisfying! This culina

In [419]:
predict(gpt_negative_reviews)

1/1 [==============================] - 0s 38ms/step
[0, 1, 67, 4, 10370, 5769, 13, 3, 1329, 582, 36, 1, 52, 0, 0, 2336, 4, 3, 1914, 0, 1691, 943, 7859, 14, 0, 6964, 32, 13181, 1901, 0, 12, 1193, 5, 7976, 1, 15356, 28, 6431, 36, 3, 10370, 0, 0, 0, 1, 5129, 6894, 7483, 874, 2146, 1731, 5047, 3, 1520, 2, 5614, 0, 16, 32, 3816, 4, 1, 13113, 10370, 0, 0, 10027, 1034, 1280, 5, 1, 0, 14, 9, 3671, 1, 4627, 0, 2, 1134, 12, 28, 19822, 8, 3, 486, 10370, 0, 0, 4, 3, 12989, 0, 1, 5769, 418, 18290, 2, 4167, 4, 1, 0, 0, 12, 141, 10179, 138, 3, 353, 0, 0, 15550, 3887, 4, 0, 2, 0, 676, 43, 5, 27, 3, 2688, 0, 13517, 114, 5, 6799, 1, 441, 0, 0, 11719, 6, 3, 3485, 210, 15, 10474, 0, 11, 840, 2217, 314, 73, 5, 27, 0, 0, 18078, 13, 8846, 0, 1197, 32, 4006, 8424, 0, 0, 3, 7523, 717, 1612, 4, 0, 42, 4736, 5, 2830, 1, 365, 2823, 4, 0, 18, 4914, 9, 5, 0, 11, 67, 4, 10370, 5769, 1193, 5, 906, 57, 1, 88, 1118, 0, 0, 18431, 1096, 12, 731, 227, 343, 4, 1, 0, 582, 28, 59, 437, 0]
Opening the can of tomato soup was a

It's hilarious how GPT spat out *"With a maximum sequence length of 200, it's challenging to encapsulate the full extent of disappointment."* and still got a positive prediction.

But as we can tell, there are many important words missing. I think using the entire corpus will improve results.

# More data == Better model?

In [434]:
(big_train, big_train_labels), (big_test, big_test_labels) = imdb.load_data(skip_top=20, start_char=1, oov_char=2)

Almost twice as many reviews as in the first dataset.

In [436]:
model2 = Sequential()
model2.add(Embedding(input_dim=88588, output_dim=128, input_length=200))
model2.add(GRU(64, dropout=0.2, recurrent_dropout=0.2))
model2.add(Dense(1, activation='sigmoid'))
model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model2.summary()

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_22 (Embedding)    (None, 200, 128)          11339264  
                                                                 
 gru_22 (GRU)                (None, 64)                37248     
                                                                 
 dense_22 (Dense)            (None, 1)                 65        
                                                                 
Total params: 11376577 (43.40 MB)
Trainable params: 11376577 (43.40 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
big_train = pad_sequences(big_train, maxlen=200)
big_test = pad_sequences(big_test, maxlen=200)

model2.fit(big_train, big_train_labels, epochs=4, batch_size=64, validation_split=0.2)
test_loss, test_acc = model2.evaluate(big_test, big_test_labels)

In [438]:
test_loss, test_acc = model2.evaluate(big_test, big_test_labels)

782/782 [==============================] - 17s 22ms/step - loss: 0.5185 - accuracy: 0.8346


In [429]:
def predict2(reviews):
    for i in range(len(reviews)):
        new_text = reviews[i]

        new_sequence = [imdb.get_word_index().get(word, 0) for word in new_text.split()]
        # new_sequence = [index if index < 20040 else 0 for index in new_sequence]
        padded = pad_sequences([new_sequence], maxlen=200)
        prediction = model2.predict(padded)
        print(prediction)

        sentiment = "Positive" if prediction[0, 0] > 0.5 else "Negative"
        print(new_sequence)
        print(reviews[i])
        print(f"Predicted sentiment: {sentiment} (Confidence: {prediction[0, 0] * 100:.2f}%)")
        print()

In [430]:
predict2(gpt_negative_reviews)

1/1 [==============================] - 0s 320ms/step
[[0.49437857]]
[0, 1, 67, 4, 10370, 5769, 13, 3, 1329, 582, 36, 1, 52, 0, 0, 2336, 4, 3, 1914, 33874, 1691, 943, 7859, 14, 0, 6964, 32, 13181, 1901, 45235, 12, 1193, 5, 7976, 1, 15356, 28, 6431, 36, 3, 10370, 0, 0, 0, 1, 5129, 6894, 7483, 874, 2146, 1731, 5047, 3, 1520, 2, 5614, 38355, 16, 32, 3816, 4, 1, 13113, 10370, 0, 0, 10027, 1034, 1280, 5, 1, 0, 14, 9, 3671, 1, 4627, 0, 2, 1134, 12, 28, 19822, 8, 3, 486, 10370, 0, 0, 4, 3, 12989, 0, 1, 5769, 418, 18290, 2, 4167, 4, 1, 32795, 39199, 12, 141, 10179, 138, 3, 353, 0, 0, 15550, 3887, 4, 31155, 2, 25155, 676, 43, 5, 27, 3, 2688, 0, 13517, 114, 5, 6799, 1, 441, 0, 0, 11719, 6, 3, 3485, 210, 15, 10474, 0, 11, 840, 2217, 314, 73, 5, 27, 0, 0, 18078, 13, 8846, 0, 1197, 32, 4006, 8424, 0, 0, 3, 7523, 717, 1612, 4, 0, 42, 4736, 5, 2830, 1, 365, 2823, 4, 0, 18, 4914, 9, 5, 0, 11, 67, 4, 10370, 5769, 1193, 5, 906, 57, 1, 88, 1118, 0, 0, 18431, 1096, 12, 731, 227, 343, 4, 1, 33874, 582, 28, 

Not as much of an improvement as I thought.

In [439]:
stop_words = []
for i in range(20):
    stop_words.append(inverted_word_index.get(i+4))

In [448]:
original_string = gpt_negative_reviews[0]
new = ' '.join(word for word in original_string.split() if word not in stop_words)
print(new)


Opening can tomato soup disappointing experience from very start. The promise delightful culinary treat quickly faded I encountered an overwhelmingly bland aroma failed evoke richness one expects from tomato soup. Upon tasting, lackluster flavor profile became painfully apparent – thin insipid broth an absence robust tomato essence. The texture further added dissatisfaction, lacked desired creaminess depth one anticipates quality tomato soup. Instead comforting consistency, soup felt watery devoid velvety smoothness should accompany such classic dish. The touted blend herbs spices turned out be mere whisper, contributing little enhance overall taste. While convenience selling point canned soups, particular product left much be desired. The aftertaste distinctly metallic, leaving an unpleasant lingering sensation. With maximum sequence length 200, it's challenging convey full extent disappointment, suffice say, can tomato soup failed meet even most basic expectations. A regrettable choi

In [449]:
predict2([new])

1/1 [==============================] - 0s 40ms/step
[[0.21261176]]
[0, 67, 10370, 5769, 1329, 582, 36, 52, 0, 0, 2336, 1914, 33874, 1691, 943, 7859, 0, 6964, 32, 13181, 1901, 45235, 1193, 7976, 15356, 28, 6431, 36, 10370, 0, 0, 0, 5129, 6894, 7483, 874, 2146, 1731, 5047, 1520, 5614, 38355, 32, 3816, 13113, 10370, 0, 0, 10027, 1034, 1280, 0, 3671, 4627, 0, 1134, 28, 19822, 486, 10370, 0, 0, 12989, 0, 5769, 418, 18290, 4167, 32795, 39199, 141, 10179, 138, 353, 0, 0, 15550, 3887, 31155, 25155, 676, 43, 27, 2688, 0, 13517, 114, 6799, 441, 0, 0, 11719, 3485, 210, 10474, 0, 840, 2217, 314, 73, 27, 0, 0, 18078, 8846, 0, 1197, 32, 4006, 8424, 0, 0, 7523, 717, 1612, 0, 42, 4736, 2830, 365, 2823, 0, 4914, 0, 67, 10370, 5769, 1193, 906, 57, 88, 1118, 0, 0, 18431, 1096, 731, 227, 343, 33874, 582, 28, 59, 437, 0]
Opening can tomato soup disappointing experience from very start. The promise delightful culinary treat quickly faded I encountered an overwhelmingly bland aroma failed evoke richness one 

In [450]:
original_string = gpt_negative_reviews[1]
new = ' '.join(word for word in original_string.split() if word not in stop_words)
predict2([new])

1/1 [==============================] - 0s 41ms/step
[[0.677312]]
[0, 18837, 5474, 0, 1030, 4659, 676, 43, 27, 18431, 0, 3715, 409, 53, 57, 88, 6108, 0, 0, 0, 1589, 2082, 27, 39581, 0, 228, 2402, 6027, 32, 16565, 0, 0, 4404, 13733, 40416, 197, 8231, 230, 0, 18837, 5474, 14078, 32, 12108, 580, 69175, 57, 88, 11958, 55403, 0, 0, 4809, 2364, 98, 5474, 0, 5945, 0, 0, 33859, 12930, 227, 36, 0, 4049, 15480, 19317, 743, 343, 0, 0, 3554, 734, 0, 18837, 5474, 2129, 6318, 4006, 0, 58605, 172, 10930, 4068, 2555, 0, 0, 1700, 486, 314, 73, 27, 0, 6453, 1338, 2119, 2838, 3274, 41, 0, 0, 71, 44813, 0, 18837, 5474, 1578, 1810, 3247, 41, 91, 20201, 0, 72077, 98, 586, 1141, 10214, 0, 0, 4404, 278, 13599, 5342, 0, 1279, 27, 0, 0, 18837, 0, 227, 36, 34553, 278, 0, 1051, 53, 4049, 4182, 1071, 197, 0, 0, 7523, 717, 1612, 0, 42, 4736, 37488, 365, 2823, 0, 4914, 0, 18837, 5474, 1193, 1642, 91, 4928, 314, 69, 7530, 2151, 4435, 0, 0, 18431, 1096, 256, 2984, 24282, 734, 39591, 0]
The tandem bike I recently purcha

By removing the stop words in the test, one of the negative reviews (Canned tomato soup review) got way better score while the other (tandem bike) got worse.

Maybe if we trained the model on food reviews instead of movies, the canned soup dilemma would've been solved.